# Wideband Dataset
This notebook showcases the Wideband dataset.

---

In [ ]:
# Define Variables

num_iq_samples_dataset = 4096 # 64^2
fft_size = 64
impairment_level = 0 # clean
num_signals_max = 5

## Wideband Metadata
In order to create a NewWideband dataset, you must define parameters in WidebandMetadata. This can be done either in code or inside a YAML file. Below we show how to do both. Look at `wideband_example.yaml` for a sample YAML file.

There are four required parameters: 
1. `num_iq_samples_dataset` -> how much IQ data per sample
2. `fft_size` -> Size of FFT (number of bins) to be used in spectrogram.
3. `impairment_level` -> what environment impairment to simulate.
4. `num_signals_max` -> maximum number of signals per sample.

Additionally, there are several optional parameters that can be overriden.

In [ ]:
# Option 1: Instantiate WidebandMetadata object
from torchsig.datasets.dataset_metadata import WidebandMetadata

wideband_metadata_1 = WidebandMetadata(
    num_iq_samples_dataset = num_iq_samples_dataset, # 64^2
    fft_size = fft_size,
    impairment_level = impairment_level, # clean
    num_signals_max = num_signals_max
)
print(wideband_metadata_1)

In [ ]:
# Option 2: Instantiate as a dictionary object

wideband_metadata_2 = dict(
    num_iq_samples = num_iq_samples_dataset,
    fft_size = fft_size,
    impairment_level = impairment_level,
    num_signals_max = num_signals_max
)
print(wideband_metadata_2)

In [ ]:
# Option 3: Instantiate from YAML file
# see wideband_example.yaml
import yaml

wideband_metadata_3_file = "wideband_example.yaml"

with open(wideband_metadata_3_file, 'r') as f:
    wideband_metadata_3 = yaml.load(f,Loader=yaml.SafeLoader)

print(wideband_metadata_3)

## Infinite Dataset
To create an infinite dataset, simply instantiate a NewWideband object. Ensure the `num_samples` field inside the WidebandMetadata is set to None.

Notes:
* You will not be able to access previously generated samples.
* You cannot save an infinite dataset to disk.

In [ ]:
from torchsig.datasets.dataset_metadata import WidebandMetadata
from torchsig.datasets.wideband import NewWideband

wideband_infinite_metadata = WidebandMetadata(
    num_iq_samples_dataset = num_iq_samples_dataset,
    fft_size = fft_size,
    impairment_level = impairment_level,
    num_signals_max = num_signals_max,
    num_samples = None
)

wideband_infinite = NewWideband(
    dataset_metadata = wideband_infinite_metadata
)
# print(wideband_infinite)

for i in range(10):
    data, metadata = wideband_infinite[i]
    if i == 5:
        print(f"IQ Data: {data.shape}")
        print(f"Signal Metadata: {metadata}")

# should fail
print()
try:
    wideband_infinite[0]
except Exception as E:
    print(E)

## Finite Dataset
To create a finite dataset, set `num_samples` = any positive integer number.

Note: you will still not be able to access previously generated samples.

In [ ]:
from torchsig.datasets.dataset_metadata import WidebandMetadata
from torchsig.datasets.wideband import NewWideband

num_samples = 10

wideband_finite_metadata = WidebandMetadata(
    num_iq_samples_dataset = num_iq_samples_dataset,
    fft_size = fft_size,
    impairment_level = impairment_level,
    num_signals_max = num_signals_max,
    num_samples = num_samples
)

wideband_finite = NewWideband(
    dataset_metadata = wideband_finite_metadata
)
# print(wideband_finite)

for i in range(11):
    if i == 10: # should fail
        try:
            data, metadata = wideband_finite[i]
        except Exception as E:
            print(E)
    else:
        data, metadata = wideband_finite[i]
        print(f"sample {i} generated successfully")

### Time Domain Plot
The default data returned from datasets are IQ samples.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

num_samples = 10

wideband_finite_time_series_metadata = WidebandMetadata(
    num_iq_samples_dataset = num_iq_samples_dataset,
    fft_size = fft_size,
    impairment_level = impairment_level,
    num_signals_max = num_signals_max,
    num_samples = num_samples,
)

wideband_finite_time_series = NewWideband(
    dataset_metadata = wideband_finite_time_series_metadata
)

data, metadata = wideband_finite_time_series[0]
t = np.arange(0,len(data))/wideband_finite_metadata.sample_rate

fig = plt.figure(figsize=(12,4))
ax = fig.add_subplot(1,1,1)
ax.plot(t,np.real(data),alpha=0.5,label='Real')
ax.plot(t,np.imag(data),alpha=0.5,label='Imag')
ax.set_xlim([t[0], t[-1]])
ax.set_xlabel('Time (sec)')
ax.set_ylabel('Amplitude')
ax.set_title('Time Domain')
ax.grid()
plt.show()


### Spectrogram Plot
Spectrograms can be generated by passing in the `Spectrogram()` transform into the metadata. For more detailed code and information on spectrogram plots, please see following section.

In [ ]:
from torchsig.transforms.transforms import Spectrogram

num_samples = 10

wideband_finite_spectrogram_metadata = WidebandMetadata(
    num_iq_samples_dataset = num_iq_samples_dataset,
    fft_size = fft_size,
    impairment_level = impairment_level,
    num_signals_max = num_signals_max,
    num_samples = num_samples,
    transforms = Spectrogram(fft_size=fft_size)
)

wideband_finite_spectrogram = NewWideband(
    dataset_metadata = wideband_finite_spectrogram_metadata
)

data, metadata = wideband_finite_spectrogram[0]

fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(1,1,1)
ax.imshow(data,aspect='auto',cmap='Wistia',vmin=0)
ax.set_xlabel('Time Axis')
ax.set_ylabel('Frequency Axis')
plt.show()

### Writing Dataset to Disk
In order to access previosuly generated examples, or save the finite dataset for later, use the `DatasetCreator`. Pass in the Dataset to be saved, where to write the dataset (root), and whether to overwrite any exisitng datasets.

In [ ]:
from torchsig.datasets.dataset_metadata import WidebandMetadata
from torchsig.datasets.wideband import NewWideband
from torchsig.utils.writer import DatasetCreator
from torchsig.signals.signal_lists import TorchSigSignalLists

root = "./datasets/wideband_example"
class_list = TorchSigSignalLists.all_signals
num_samples = len(class_list) * 10
seed = 123456789

wideband_finite_metadata = WidebandMetadata(
    num_iq_samples_dataset = num_iq_samples_dataset,
    fft_size = fft_size,
    impairment_level = impairment_level,
    num_signals_max = num_signals_max,
    num_samples = num_samples,
    seed = seed
)

wideband_finite = NewWideband(
    dataset_metadata = wideband_finite_metadata
)

dataset_creator = DatasetCreator(
    dataset = wideband_finite,
    root = root,
    # overwrite = True,
    # increase the batch size and num_workers to speed up
    # batch_size = 16,
    # num_workers = 16
)

dataset_creator.create()

### Reading Dataset from Disk
Assuming you wrote a dataset to disk, you can load it back in my instantiating a `StaticWideband`.

Warning: The following code assumes you have the code in the section **Writing Dataset to Disk**

In [ ]:
from torchsig.datasets.wideband import StaticWideband

static_wideband = StaticWideband(
    root = root,
    impairment_level = impairment_level,
)

# can access any sample
print(static_wideband[0])
print(static_wideband[5])

If the dataset writtent to disk is raw (aka no Transforms or Target Transforms were applied to it before writing to disk), then you can define whatever transforms and target transforms inside the StaticWideband.

In [ ]:
print(f"Dataset Raw: {static_wideband.raw}")

In [ ]:
from torchsig.datasets.wideband import StaticWideband
from torchsig.transforms.transforms import Spectrogram
from torchsig.transforms.target_transforms import (
    ClassName,
    Start,
    Stop,
    LowerFreq,
    UpperFreq,
    SNR
)
import matplotlib.pyplot as plt

target_transform = [
    ClassName(),
    Start(),
    Stop(),
    LowerFreq(),
    UpperFreq(),
    SNR()
]

static_wideband = StaticWideband(
    root = root,
    impairment_level = impairment_level,
    transforms = Spectrogram(fft_size = fft_size),
    target_transforms = target_transform,
)
sample_rate = static_wideband.dataset_metadata.sample_rate
noise_power_db = static_wideband.dataset_metadata.noise_power_db

num_show = 4
num_cols = 2
num_rows = num_show // num_cols

fig = plt.figure(figsize=(18,12))
fig.tight_layout()

max_time = num_iq_samples_dataset / sample_rate

ns_time = 1e-9
us_time = 1e-6
ms_time = 1e-3
s_time = 1

if (max_time < ns_time):
    max_time /= ns_time
    xunits = 'ns'
elif (max_time < us_time):
    max_time /= us_time
    xunits = 'us'
elif (max_time < ms_time):
    max_time /= ms_time
    xunits = 'ms'
else:
    max_time /= s_time
    xunits = 's'

xmin = 0 * max_time
xmax = 1 * max_time

ymin = -sample_rate / 2
ymax = sample_rate / 2

for i in range(num_show):

    data, targets = static_wideband[i]
    ax = fig.add_subplot(num_rows,num_cols,i + 1)

    pos = ax.imshow(data,extent=[xmin,xmax,ymin,ymax],aspect='auto',cmap='Wistia',vmin=noise_power_db)
    fig.colorbar(pos, ax=ax, label='SNR')

    for t in targets:
        classname, start, stop, lower, upper, snr = t

        # convert normalized time into real-world time
        start_time = start * max_time
        stop_time = stop * max_time

        ax.plot([start_time,start_time],[lower,upper],'b',alpha=0.5)
        ax.plot([stop_time, stop_time],[lower,upper],'b',alpha=0.5)
        ax.plot([start_time,stop_time],[lower,lower],'b',alpha=0.5)
        ax.plot([start_time,stop_time],[upper,upper],'b',alpha=0.5)
        textDisplay = str(classname) + ', SNR = ' + str(snr) + ' dB'
        ax.text(start_time,lower,textDisplay, bbox=dict(facecolor='w', alpha=0.5, linewidth=0))
        ax.set_xlim([xmin,xmax])
        ax.set_ylim([ymin,ymax])
        ax.set_ylabel("Frequency (Hz)")
        ax.set_xlabel(f"Time ({xunits})")


## Wideband Dataset Statistics

Below are some plots and statistics about the Wideband dataset.

In [ ]:
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

class_list = static_wideband.dataset_metadata.class_list
class_counter = {class_name: 0 for class_name in class_list}
snr_list = []
num_signals_per_sample = []

for sample in tqdm(static_wideband, desc = "Calculating Wideband Stats"):
    data, targets = sample
    num_signals_per_sample.append(len(targets))
    for t in targets:
        classname, start, stop, lower, upper, snr = t
        class_counter[classname] += 1
        snr_list.append(snr)

class_counts = list(class_counter.values())
class_names = list(class_counter.keys())


In [ ]:
# Class Distribution Pie Chart
# by default, the class distribution is None aka uniform
print(f"Class Distribution Setting: {static_wideband.dataset_metadata.class_distribution}")
plt.figure(figsize=(15, 15))
plt.pie(class_counts, labels = class_names)
plt.title("Class Distribution Pie Chart")
plt.show()

In [ ]:
# Class Distribution Bar Chart
plt.figure(figsize=(18, 4))
plt.bar(class_names, class_counts)
plt.xticks(rotation=90)
plt.title("Class Distribution Bar Chart")
plt.xlabel("Modulation Class Name")
plt.ylabel("Counts")
plt.show()

In [ ]:
# Number of signals per sample Distribution
import numpy as np
# Wideband's default number of signals per sample is 3
print(f"Min num signals Setting: {static_wideband.dataset_metadata.num_signals_min}")
print(f"Max num signals Setting: {static_wideband.dataset_metadata.num_signals_max}")

total = sum(num_signals_per_sample)
avg = np.mean(np.asarray(num_signals_per_sample))

plt.figure(figsize=(11,8))
plt.hist(x=num_signals_per_sample, bins=np.arange(max(num_signals_per_sample) + 1) + 0.5)
plt.xticks(np.arange(0, max(num_signals_per_sample) + 1, 1).tolist())
plt.title(f"Distribution of Number of Signals Per Sample\nTotal Number: {total}, Average: {avg}")
plt.xlabel("Number of Signal Bins")
plt.ylabel("Counts")
plt.show()

In [ ]:
# SNR Distributions
# Default min = 0 db and max = 50 db
print(f"Min SNR Setting: {static_wideband.dataset_metadata.snr_db_min}")
print(f"Max SNR Setting: {static_wideband.dataset_metadata.snr_db_max}")
plt.figure(figsize=(11, 4))
plt.hist(x=snr_list, bins=100)
plt.title("SNR Distribution")
plt.xlabel("SNR Bins (dB)")
plt.ylabel("Counts")
plt.show()